In [ ]:
! cp -a ../input/depend/. ./

In [ ]:
# update the libraries
import os
for path, _, files in os.walk('./'):
    for file in files:
        file_dir = os.path.join(path, file)
        if 'xyz' in file_dir:
            print(file_dir)
            new_path = file_dir.replace('xyz', 'tar.gz')
            cmd = f'mv {file_dir} {new_path}'
            ! {cmd}

In [ ]:
! rm ./mmcv1317/mmcv-full-1.3.17.tar.gz
! cp ../input/mmcv1317whl/mmcv_full-1.3.17-cp37-cp37m-linux_x86_64.whl ./mmcv1317/
!ls ./mmcv1317

In [ ]:
!pip install ../input/work-dir/pytest_runner-5.3.1-py3-none-any.whl
!pip install --no-index --find-links ./addict240 addict
!pip install --no-index --find-links ./pycocotools pycocotools
!pip install --no-index --find-links ./yapf0310 yapf
!pip install --no-index --find-links ./mmdet2180 mmdet
!pip install --no-index --find-links ./mmcv1317 mmcv-full

In [ ]:
import sys
import pandas as pd
import numpy as np
# import cupy as cp
from glob import glob
import os
import cv2
import gc
from tqdm.notebook import tqdm
import pickle
from itertools import groupby
from pycocotools import mask as mutils
from pycocotools import _mask as coco_mask
import matplotlib.pyplot as plt
import os
import base64
import typing as t
import zlib
import random
random.seed(0)

In [ ]:
class Cfg:
    # Library versions
    ADDICT_VER = "2.4.0"
    MMCV_VER = "1.3.17"
    MMDETECTION_VER = "2.18.0"
    PYCOCOTOOLS_VER = "1.0.0"
    YAPF_VER = "0.31.0"
    # Config name
    MODEL_CFG = "maskrcnn_resnet101"
    # Which checkout to be used?
    CHECKPOINT = "epoch_23"
    # How many samples it visualizes
    NUM_VISUALIZE_SAMPLES = 3
    TRAIN_OR_TEST = "test"
    # Dataset directory
    DATA_DIR = "../input/sartorius-cell-instance-segmentation"
    # Rirectory which contains numpy converted masks
    MASK_DIR = "masks"
    # ???
    IMG_DIR = f"imgs"
    # Leave the bboxes which score is larger than this value
    SCORE_TH = 0.3
    FOLD = 0
    # Train data ratio
    TRAIN_RATIO = 0.95
    NUM_GPUS = 1
    FIRST_RUN = True
    
    
CFG = Cfg()

In [ ]:
! mkdir ./cfg

## Generate custom dataset for test

In [ ]:
def load_meta_info(cfg):
    meta_info_path = os.path.join(cfg.DATA_DIR, f"{cfg.TRAIN_OR_TEST}/*")
    print(f"Loading file names from {meta_info_path}")
    df  = pd.DataFrame(glob(meta_info_path), columns=["image_path"])
    df["id"] = df.image_path.map(lambda x: x.split("/")[-1].split(".")[0])
    print(f"Loaded {len(df)} file names")
    display(df.head())
    return df


df = load_meta_info(CFG)

In [ ]:
def read_img(cfg, image_id, image_size=None):
    filename = os.path.join(f"{cfg.DATA_DIR}", f"{cfg.TRAIN_OR_TEST}", f"{image_id}.png")
    assert os.path.exists(filename)
    img = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
    if image_size is not None:
        img = cv2.resize(img, (image_size, image_size))
    if img.dtype == "uint16":
        img = (img/256).astype("uint8")
    return img

def print_masked_img(cfg, image_id, mask):
    img   = read_img(cfg, image_id)
    print(img.shape)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    img2  = clahe.apply(img)
    img3  = cv2.equalizeHist(img)
    img   = np.stack([img, img2, img3],axis=-1) 
    plt.figure(figsize=(15, 15))
    plt.subplot(1, 3, 1)
    plt.imshow(img)
    plt.title("Image")
    plt.axis("off")
    plt.subplot(1, 3, 2)
    plt.imshow(mask,cmap="inferno")
    plt.title("Mask")
    plt.axis("off")   
    plt.subplot(1, 3, 3)
    plt.imshow(img)
    plt.imshow(mask, alpha=0.4, cmap="inferno")
    plt.title("Image + Mask")
    plt.axis("off")
    plt.tight_layout()
    plt.show()

In [ ]:
def pickle_test_anns(cfg, df):
    out_image_dir = f"{cfg.TRAIN_OR_TEST}/"
    !mkdir -p {out_image_dir}
    test_anns = []
    for idx in tqdm(range(len(df))):
        image_id = df.iloc[idx]["id"]
        img = read_img(cfg, image_id)
        test_ann = {
            "filename": image_id + ".png",
            "width": img.shape[1],
            "height": img.shape[0],
            "ann": {
                "bboxes": None,
                "labels": None,
                "masks": None
            }
        }
        test_anns.append(test_ann)
    with open(f"{cfg.TRAIN_OR_TEST}/test_anns.pkl", "wb") as f:
        pickle.dump(test_anns, f)
    return test_anns
        
        
test_anns = pickle_test_anns(CFG, df)

In [ ]:
%%writefile ./cfg/{Cfg.MODEL_CFG}.py
model = dict(
    type='MaskRCNN',
    backbone=dict(
        type='ResNet',
        depth=101,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained',
                      checkpoint='torchvision://resnet101')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        loss_bbox=dict(type='L1Loss', loss_weight=1.0)),
    roi_head=dict(
        type='StandardRoIHead',
        bbox_roi_extractor=dict(
            type='SingleRoIExtractor',
            roi_layer=dict(type='RoIAlign', output_size=7, sampling_ratio=0),
            out_channels=256,
            featmap_strides=[4, 8, 16, 32]),
        bbox_head=dict(
            type='Shared2FCBBoxHead',
            in_channels=256,
            fc_out_channels=1024,
            roi_feat_size=7,
            num_classes=3,
            bbox_coder=dict(
                type='DeltaXYWHBBoxCoder',
                target_means=[0.0, 0.0, 0.0, 0.0],
                target_stds=[0.1, 0.1, 0.2, 0.2]),
            reg_class_agnostic=False,
            loss_cls=dict(
                type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0),
            loss_bbox=dict(type='L1Loss', loss_weight=1.0)),
        mask_roi_extractor=dict(
            type='SingleRoIExtractor',
            roi_layer=dict(type='RoIAlign', output_size=14, sampling_ratio=0),
            out_channels=256,
            featmap_strides=[4, 8, 16, 32]),
        mask_head=dict(
            type='FCNMaskHead',
            num_convs=4,
            in_channels=256,
            conv_out_channels=256,
            num_classes=3,
            loss_mask=dict(
                type='CrossEntropyLoss', use_mask=True, loss_weight=1.0))),
    train_cfg=dict(
        rpn=dict(
            assigner=dict(
                type='MaxIoUAssigner',
                pos_iou_thr=0.7,
                neg_iou_thr=0.3,
                min_pos_iou=0.3,
                match_low_quality=True,
                ignore_iof_thr=-1),
            sampler=dict(
                type='RandomSampler',
                num=256,
                pos_fraction=0.5,
                neg_pos_ub=-1,
                add_gt_as_proposals=False),
            allowed_border=-1,
            pos_weight=-1,
            debug=False),
        rpn_proposal=dict(
            nms_pre=2000,
            max_per_img=1000,
            nms=dict(type='nms', iou_threshold=0.7),
            min_bbox_size=0),
        rcnn=dict(
            assigner=dict(
                type='MaxIoUAssigner',
                pos_iou_thr=0.5,
                neg_iou_thr=0.5,
                min_pos_iou=0.5,
                match_low_quality=True,
                ignore_iof_thr=-1),
            sampler=dict(
                type='RandomSampler',
                num=512,
                pos_fraction=0.25,
                neg_pos_ub=-1,
                add_gt_as_proposals=True),
            mask_size=28,
            pos_weight=-1,
            debug=False)),
    test_cfg=dict(
        rpn=dict(
            nms_pre=1000,
            max_per_img=1000,
            nms=dict(type='nms', iou_threshold=0.7),
            min_bbox_size=0),
        rcnn=dict(
            score_thr=0.05,
            nms=dict(type='nms', iou_threshold=0.5),
            max_per_img=100,
            mask_thr_binary=0.5)))
dataset_type = 'CocoDataset'
data_root = '../input/sartorius-cell-instance-segmentation'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)

data = dict(
    samples_per_gpu=2,
    workers_per_gpu=2,
    train=dict(
        type='CocoDataset',
        ann_file=data_root+
        '../input/coco-format/train_coco_fold_0.json',
        img_prefix=data_root,
        pipeline=[
            dict(type='LoadImageFromFile', to_float32=True),
            dict(type='LoadAnnotations', with_bbox=True, with_mask=True),
            dict(type='Resize', img_scale=(512, 512), keep_ratio=True),
            dict(type='RandomFlip', flip_ratio=0.5),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_rgb=True),
            dict(type='Pad', size_divisor=32),
            dict(type='DefaultFormatBundle'),
            dict(
                type='Collect',
                keys=['img', 'gt_bboxes', 'gt_labels', 'gt_masks'])
        ],
        classes=('cort', 'shsy5y', 'astro')),
    val=dict(
        type='CocoDataset',
        ann_file=
        '../input/coco-format/val_coco_fold_0.json',
        img_prefix=data_root,
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(
                type='MultiScaleFlipAug',
                img_scale=(512, 512),
                flip=False,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='ImageToTensor', keys=['img']),
                    dict(type='Collect', keys=['img'])
                ])
        ],
        classes=('cort', 'shsy5y', 'astro')),
    test=dict(
        type='CustomDataset',
        ann_file='./test/test_anns.pkl',
        img_prefix=data_root+'/test',
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(
                type='MultiScaleFlipAug',
                img_scale=(512, 512),
                flip=False,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='ImageToTensor', keys=['img']),
                    dict(type='Collect', keys=['img'])
                ])
        ],
        classes=('cort', 'shsy5y', 'astro')))
evaluation = dict(metric=['bbox', 'segm'])
optimizer = dict(type='SGD', lr=0.02, momentum=0.9, weight_decay=0.0001)
optimizer_config = dict(grad_clip=None)
lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=500,
    warmup_ratio=0.001,
    step=[16, 22])
runner = dict(type='EpochBasedRunner', max_epochs=24)
checkpoint_config = dict(interval=1)
log_config = dict(interval=50, hooks=[dict(type='TextLoggerHook')])
custom_hooks = [dict(type='NumClassCheckHook')]
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = None
resume_from = None
workflow = [('train', 1)]
classes = ('cort', 'shsy5y', 'astro')
work_dir = './test/mask_rcnn_save'
gpu_ids = range(0, 1)


## Test process


In [ ]:
! ls ./cfg/

In [ ]:
def run_test_script(cfg):
    config = f"./cfg/{cfg.MODEL_CFG}.py"
    model_file = f"../input/saved-weights/{cfg.CHECKPOINT}.pth"
    result_pkl = f"test/{cfg.MODEL_CFG}.pkl"
    additional_conf = "--cfg-options"
    additional_conf += f" model.test_cfg.rcnn.score_thr={cfg.SCORE_TH}"
    cmd = f"python ../input/mmdet2180/mmdetection-2.18.0/tools/test.py {config} {model_file} --out {result_pkl} {additional_conf}"
    !{cmd}
    result = pickle.load(open(result_pkl, "rb"))
    return result


test_results = run_test_script(CFG)

In [ ]:

def show_results(cfg, anns, results):
    num_imgs = len(results)
    for i in range(num_imgs):
        image_id = anns[i]["filename"].replace(".jpg", "").replace(".png", "")
        print(image_id)
        for class_id in range(3):
            bbs = results[i][0][class_id]
            sgs = results[i][1][class_id]
            if len(sgs) == 0:
                continue
            for idx, (bb, sg) in enumerate(zip(bbs, sgs)):
                box = bb[:4]
                cnf = bb[4]
                h = sg["size"][0]
                w = sg["size"][0]
                if cnf > 0.1:
                    if idx == 0:
                        mask = mutils.decode(sg)
                    else:
                        mask += mutils.decode(sg)
            print(mask.shape)
            print_masked_img(cfg, image_id, mask)
            
            
show_results(CFG, test_anns, test_results)

In [ ]:
def mask2rle(mask):
    mask   = np.array(mask)
    pixels = mask.flatten()
    pad    = np.array([0])
    pixels = np.concatenate([pad, pixels, pad])
    runs   = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(str(x) for x in runs)

def one_hot(y, num_classes, dtype=np.uint8):
    y = np.array(y, dtype="int")
    input_shape = y.shape
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])
    y = y.ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes), dtype=dtype)
    categorical[np.arange(n), y] = 1
    output_shape = input_shape + (num_classes,)
    categorical = np.reshape(categorical, output_shape)
    return categorical


def fix_overlap(msk): # GPU
    msk = np.array(msk)
    msk = np.pad(msk, [[0, 0], [0, 0], [1, 0]])
    ins_len = msk.shape[-1]
    msk = np.argmax(msk,axis=-1)
    msk = one_hot(msk, num_classes=ins_len, )
    msk = msk[..., 1:]
    msk = msk[..., np.any(msk, axis=(0,1))]
    return msk


def check_overlap(msk):
    msk = msk.astype(bool).astype(np.uint8)
    return np.any(np.sum(msk, axis=-1) > 1)

In [ ]:
data = []
for ii in tqdm(range(len(test_anns))):
    image_id = test_anns[ii]["filename"].replace(".jpg","").replace(".png","")
    mask = []
    for class_id in range(3):
        bbs = test_results[ii][0][class_id]
        sgs = test_results[ii][1][class_id]
        if len(sgs) == 0:
            continue
        
        for bb, sg in zip(bbs,sgs):
            box = bb[:4]
            cnf = bb[4]
            h = sg["size"][0]
            w = sg["size"][1]
            #convert coco format to kaggle format
            mask.append(np.array(mutils.decode(sg)))
    mask = np.stack(mask, axis=-1)
    if check_overlap(mask): # if mask instances have overlap then fix it
        mask = fix_overlap(mask)
    if check_overlap(mask):
        print("still overlap")
    p_mask = np.amax(mask, axis=-1)
    print(p_mask.shape)
    print_masked_img(CFG, image_id, p_mask)
    for idx in range(mask.shape[-1]):
        mask_ins = mask[..., idx]
        rle  = mask2rle(mask_ins)
        data.append([image_id, rle])
    del mask, rle, sgs, bbs
    gc.collect()
pred_df = pd.DataFrame(data, columns=["id", "predicted"])

In [ ]:
sub_df  = pd.read_csv("../input/sartorius-cell-instance-segmentation/sample_submission.csv")
del sub_df["predicted"]
sub_df = sub_df.merge(pred_df, on="id", how="left")
sub_df.to_csv("submission.csv", index=False)
sub_df.head()

In [ ]:
! rm -rf ./mmdet2180/
! rm -rf ./yapf0310/
! rm -rf ./test/
! rm -rf ./pycocotools/
! rm -rf ./addict240/
! rm -rf ./mmcv1317/
! rm -rf ./cfg

In [ ]:
! ls -lah ./